In [1]:
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import SFTConfig, SFTTrainer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [2]:
dataset_path = 'data/eminem_lyrics_prompt_completion'
dataset = load_from_disk(dataset_path)

In [3]:
dataset = dataset.remove_columns('track_name')

In [4]:
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Users\HP\.conda\envs\llmgpu\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
training_args = SFTConfig(
    packing=False,
    output_dir='./results',  # Directory to save the model
    logging_dir='./logs',    # Directory to save logs
    logging_steps=10,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    fp16=True
)

In [6]:
def custom_formatting_func(example):
    prompt = example['prompt']
    completion = example['completion']
    text = prompt + completion
    return text

In [7]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    formatting_func=custom_formatting_func
)


c:\Users\HP\.conda\envs\llmgpu\Lib\site-packages\trl\trainer\sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1285 [00:00<?, ? examples/s]

c:\Users\HP\.conda\envs\llmgpu\Lib\site-packages\trl\trainer\sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


In [8]:
trainer.train()

trainer.save_model('./trained_model')
trainer.save_state()

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/artaasd95/general/ed1fdc5113304bfab3f0e81e4b84bc1f



  0%|          | 0/963 [00:00<?, ?it/s]

COMET WARNING: Unknown error exporting current conda environment
COMET WARNING: Unknown error retrieving Conda package as an explicit file
COMET WARNING: Unknown error retrieving Conda information
c:\Users\HP\.conda\envs\llmgpu\Lib\site-packages\torch\autograd\graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\MHA.cpp:676.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


{'loss': 16.7378, 'grad_norm': 25.70503044128418, 'learning_rate': 4.994807892004154e-05, 'epoch': 0.03}
{'loss': 15.5545, 'grad_norm': 17.0876522064209, 'learning_rate': 4.9480789200415374e-05, 'epoch': 0.06}
{'loss': 14.7086, 'grad_norm': 30.798337936401367, 'learning_rate': 4.9065420560747664e-05, 'epoch': 0.09}
{'loss': 14.7624, 'grad_norm': 56.66119384765625, 'learning_rate': 4.8546209761163035e-05, 'epoch': 0.12}
{'loss': 15.2694, 'grad_norm': 40.317684173583984, 'learning_rate': 4.80269989615784e-05, 'epoch': 0.16}
{'loss': 14.4289, 'grad_norm': 58.76542663574219, 'learning_rate': 4.750778816199377e-05, 'epoch': 0.19}
{'loss': 15.6919, 'grad_norm': 38.806854248046875, 'learning_rate': 4.7040498442367604e-05, 'epoch': 0.22}
{'loss': 14.4302, 'grad_norm': 42.680877685546875, 'learning_rate': 4.6521287642782976e-05, 'epoch': 0.25}
{'loss': 14.7392, 'grad_norm': 30.57004737854004, 'learning_rate': 4.60539979231568e-05, 'epoch': 0.28}
{'loss': 14.69, 'grad_norm': 46.240089416503906, 

c:\Users\HP\.conda\envs\llmgpu\Lib\site-packages\torch\autograd\graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\MHA.cpp:676.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


{'loss': 20.7324, 'grad_norm': 87.36784362792969, 'learning_rate': 2.4662512980269992e-05, 'epoch': 1.59}
{'loss': 21.8107, 'grad_norm': 87.93720245361328, 'learning_rate': 2.414330218068536e-05, 'epoch': 1.62}
{'loss': 22.6019, 'grad_norm': 71.55953216552734, 'learning_rate': 2.367601246105919e-05, 'epoch': 1.65}
{'loss': 22.1661, 'grad_norm': 112.86392974853516, 'learning_rate': 2.315680166147456e-05, 'epoch': 1.68}
{'loss': 22.4631, 'grad_norm': 81.55806732177734, 'learning_rate': 2.263759086188993e-05, 'epoch': 1.71}
{'loss': 22.4102, 'grad_norm': 78.8140869140625, 'learning_rate': 2.2118380062305298e-05, 'epoch': 1.74}
{'loss': 22.2649, 'grad_norm': 78.72042846679688, 'learning_rate': 2.1599169262720665e-05, 'epoch': 1.77}
{'loss': 20.6629, 'grad_norm': 77.41638946533203, 'learning_rate': 2.1079958463136033e-05, 'epoch': 1.81}
{'loss': 21.2536, 'grad_norm': 76.36277770996094, 'learning_rate': 2.05607476635514e-05, 'epoch': 1.84}
{'loss': 22.9393, 'grad_norm': 84.59651184082031, 'l

In [9]:
# Load the trained model and tokenizer
model_path = './trained_model'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Generate Rap Lyrics
prompt = "generate Rap lyrics like this lyrics:\nLook, I was gonna go easy on you not to hurt your feelings\nBut I'm only going to get this one chance "
inputs = tokenizer.encode(prompt, return_tensors="pt")
outputs = model.generate(inputs, max_length=200, do_sample=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


generate Rap lyrics like this lyrics:
Look, I was gonna go easy on you not to hurt your feelings
But I'm only going to get this one chance 
Dick I got a shit my way with that
Thit dont the me I got to try but so try em?
You are when I cant come this up your motherfucker?
You dont even know you dont just leave my motherfuckers
I gotta say you dont even look up in the up this, I got out, they are never to get in a big time and me Im like, we want the em to fuck the man, fuck a fuck I see that, I know shit, you dont think you still get your shit right in the man, I say you see my way but they dont get my man, you wont know what I go and thats with a bitch, it really got up but it dont feel in more and shit it dont know what you think you got a boy
